In [12]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import os
import requests
import json
#change to appropriate home folder
folder = os.getcwd() + '/'

In [2]:
from openai import AzureOpenAI
import os
import openai
from openai import OpenAI

In [9]:
def extract_ade_terms_gpt4(text):
  
  AZURE_OPENAI_KEY = "1cc983d783784fcf9564848ea6cb7cc4"
  AZURE_OPENAI_ENDPOINT = "https://onsides-gpt-finetuning.openai.azure.com/"
    
  client = AzureOpenAI(
      api_key = AZURE_OPENAI_KEY,  
      api_version = "2023-05-15",
      azure_endpoint = AZURE_OPENAI_ENDPOINT
    )

  chat_completion = client.chat.completions.create(
      messages=[
          {"role": "system", "content": "You are an expert in pharmacology."},
          {
              "role": "user",
              "content": "The following is an excerpt of a drug label. "\
          "Your task is to find the adverse drug event terms in this text. "\
          "Maximize RECALL if possible." \
          "Return only the terms as a comma-separated list. "\
          "The input is {}".format(text)
          }
      ],
      model="gpt-35-finetuned-bottomlabels"
  )
  term = chat_completion.choices[0].message.content
  return term

---

---

---

In [4]:
train_drug = pd.read_csv(folder+'train_drug_label_text_remove_unnecessary_info.csv')
train_drug.head(1)

,drug_name,section_name,section_text,section_text_removed
0,KYPROLIS,adverse reactions,6 ADVERSE REACTIONS\n\n The following adv...,The following adverse reactions are discussed ...


In [10]:
gpt4_output = []
for i, row in tqdm(train_drug.iterrows()):
  name, section= row['drug_name'], row['section_name']
  text = row['section_text']
  try:
    gpt_out = extract_ade_terms_gpt4(text)
  except:
    gpt_out = None
  gpt4_output.append([name, section, gpt_out])
gpt4_output = pd.DataFrame(gpt4_output, columns=['drug_name', 'section_name', 'gpt4_output'])
gpt4_output.to_csv(folder+'train_drug_label_gpt35_finetuned_output_3.csv', index= False)
gpt4_output.head(1)

239it [06:03,  1.52s/it]


,drug_name,section_name,gpt4_output
0,KYPROLIS,adverse reactions,None


In [6]:
test_df = pd.read_csv(folder+'train_drug_label_gpt35_finetuned_output.csv')
test_df.head(10)

,drug_name,section_name,gpt4_output
0,KYPROLIS,adverse reactions,NaN
1,KYPROLIS,warnings and precautions,"Cardiac failure, myocardial infarction, myocar..."
2,MULTAQ,adverse reactions,"New or worsening heart failure, Liver Injury, ..."
3,MULTAQ,boxed warnings,"increased risk of death, stroke, heart failure..."
4,MULTAQ,warnings and precautions,"Cardiac rhythm, AF, hepatic injury, pulmonary ..."
5,JUBLIA,adverse reactions,"ingrown toenails, application site dermatitis,..."
6,TEFLARO,adverse reactions,"hypersensitivity reactions, clostridium diffic..."
7,TEFLARO,warnings and precautions,"hypersensitivity reactions, anaphylactic react..."
8,DATSCAN,adverse reactions,"Hypersensitivity, injection site reactions, he..."
9,DATSCAN,warnings and precautions,"Hypersensitivity reactions, anaphylactic, hype..."


In [7]:
test_drug = pd.read_csv(folder+'test_drug_label_text_remove_unnecessary_info.csv')
test_drug.head(1)

,drug_name,section_name,section_text,section_text_removed
0,IMPAVIDO,adverse reactions,6 ADVERSE REACTIONS\n\n Because clinical ...,EXCERPT: * Adverse reactions occurring i...


In [8]:
gpt4_output = []
for i, row in tqdm(test_drug.iterrows()):
  name, section = row['drug_name'], row['section_name']
  text = row['section_text']
  try:
    gpt_out = extract_ade_terms_gpt4(text)
  except:
    gpt_out = None
  gpt4_output.append([name, section, gpt_out])
gpt4_output = pd.DataFrame(gpt4_output, columns=['drug_name', 'section_name', 'gpt4_output'])
gpt4_output.to_csv(folder+'test_drug_label_gpt35_finetuned_output_2.csv', index= False)
gpt4_output.head(1)

237it [05:18,  1.34s/it]


,drug_name,section_name,gpt4_output
0,IMPAVIDO,adverse reactions,"nausea, vomiting, diarrhea, headache, decrease..."


----